In [16]:
# Importing the required libraries 
import os
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from pyculiarity import detect_ts

# Load Meter and Meta Data

In [2]:
all_meters = pd.read_csv("../data/meters/cleaned/allmeters_daily.csv")
meta_data = pd.read_csv("../data/metadata/metadata.csv")

We are only interested looking at Energy Consumption by 
- Educational Institutions
- Office

### Process metadata buiding

In [3]:
#meta_education = meta_data[meta_data['primaryspaceusage'] == 'Education']
# meta_education
meta_education_office = meta_data[meta_data['primaryspaceusage'].isin(['Education', 'Office'])]
meta_education_office

,building_id,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,lat,lng,...,yearbuilt,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating
3,Panther_education_Rosalie,Panther,0.0,0.0,Education,Research,690.5,7432.0,28.517689,-81.379039,...,2008.0,NaN,NaN,NaN,NaN,276,NaN,NaN,NaN,NaN
4,Panther_education_Misty,Panther,1.0,0.0,Education,Research,252.7,2720.0,28.517689,-81.379039,...,2004.0,NaN,NaN,NaN,NaN,375,NaN,NaN,NaN,NaN
5,Panther_office_Daina,Panther,NaN,0.0,Office,Office,133.8,1440.0,28.517689,-81.379039,...,1983.0,NaN,NaN,NaN,NaN,1983,NaN,NaN,NaN,NaN
6,Panther_education_Mattie,Panther,2.0,0.0,Education,Classroom,499.4,5376.0,28.517689,-81.379039,...,1991.0,NaN,NaN,NaN,NaN,49,NaN,NaN,NaN,NaN
7,Panther_office_Woodrow,Panther,NaN,0.0,Office,Office,537.8,5789.0,28.517689,-81.379039,...,1996.0,NaN,NaN,NaN,NaN,136,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,Cockatoo_education_Collin,Cockatoo,1439.0,15.0,Education,College Laboratory,5248.7,56497.0,42.459837,-76.485292,...,1957.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1621,Cockatoo_education_Gussie,Cockatoo,1441.0,15.0,Education,College Laboratory,2800.4,30143.0,42.459837,-76.485292,...,1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1623,Cockatoo_education_Arlen,Cockatoo,1443.0,15.0,Education,College Classroom,3745.0,40311.0,42.459837,-76.485292,...,1913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1625,Cockatoo_education_Victoria,Cockatoo,1445.0,15.0,Education,College Classroom,399.3,4298.0,42.459837,-76.485292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have 924 buildings that were used for Educational and Office purposes

In [4]:
meta_education_office.info()

<class 'pandas.core.frame.DataFrame'>
Index: 924 entries, 3 to 1628
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   building_id            924 non-null    object 
 1   site_id                924 non-null    object 
 2   building_id_kaggle     828 non-null    float64
 3   site_id_kaggle         915 non-null    float64
 4   primaryspaceusage      924 non-null    object 
 5   sub_primaryspaceusage  924 non-null    object 
 6   sqm                    924 non-null    float64
 7   sqft                   924 non-null    float64
 8   lat                    827 non-null    float64
 9   lng                    827 non-null    float64
 10  timezone               924 non-null    object 
 11  electricity            900 non-null    object 
 12  hotwater               117 non-null    object 
 13  chilledwater           371 non-null    object 
 14  steam                  235 non-null    object 
 15  water     

Base on the EDA, and the number of missing values, the following fields in metadata will be kept:
1. building_id
2. site_id
3. primaryspaceusage
4. sqm
5. sqft
6. timezone
7.  



In [5]:
# Since not all buildings use all energy sources, marking the sources used for each building 
meter_columns = ['electricity', 'hotwater', 'chilledwater', 'steam', 'water', 'irrigation', 'solar', 'gas']
for col in meter_columns:
    meta_education_office[col] = np.where(meta_education_office[col] == 'Yes', 1, 0)   # where the meter is present, mark 1, else 0

/tmp/ipykernel_32197/118997712.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_education_office[col] = np.where(meta_education_office[col] == 'Yes', 1, 0)   # where the meter is present, mark 1, else 0


In [6]:
# Melt the DataFrame to pivot it
meta_melted = meta_education_office.melt(id_vars=[col for col in meta_education_office.columns if col not in meter_columns], 
                    value_vars=meter_columns, 
                    var_name='meter', 
                    value_name='value')

# Filtering for the corrrect 
meta_melted = meta_melted[meta_melted['value']==1]

In [7]:
# Merging both the dataframes together
merged_df = pd.merge(all_meters, meta_melted, on=['building_id','meter' ], how='inner')

In [8]:
merged_df.columns

Index(['building_id', 'meter', 'date', 'meter_reading', 'site_id',
       'building_id_kaggle', 'site_id_kaggle', 'primaryspaceusage',
       'sub_primaryspaceusage', 'sqm', 'sqft', 'lat', 'lng', 'timezone',
       'industry', 'subindustry', 'heatingtype', 'yearbuilt', 'date_opened',
       'numberoffloors', 'occupants', 'energystarscore', 'eui', 'site_eui',
       'source_eui', 'leed_level', 'rating', 'value'],
      dtype='object')

In [9]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1343578 entries, 0 to 1343577
Data columns (total 28 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   building_id            1343578 non-null  object 
 1   meter                  1343578 non-null  object 
 2   date                   1343578 non-null  object 
 3   meter_reading          1343578 non-null  float64
 4   site_id                1343578 non-null  object 
 5   building_id_kaggle     1238314 non-null  float64
 6   site_id_kaggle         1334075 non-null  float64
 7   primaryspaceusage      1343578 non-null  object 
 8   sub_primaryspaceusage  1343578 non-null  object 
 9   sqm                    1343578 non-null  float64
 10  sqft                   1343578 non-null  float64
 11  lat                    1144746 non-null  float64
 12  lng                    1144746 non-null  float64
 13  timezone               1343578 non-null  object 
 14  industry          

In [10]:
merged_df.sample(10)

,building_id,meter,date,meter_reading,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,...,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating,value
272171,Cockatoo_education_Doreen,electricity,2016-08-27,2188.6750,Cockatoo,1356.0,15.0,Education,College Classroom,8477.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1212964,Rat_education_Mose,electricity,2016-08-23,3526.7900,Rat,412.0,3.0,Education,K-12 School,10219.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
921603,Lamb_education_Patsy,electricity,2017-06-27,0.0072,Lamb,NaN,5.0,Education,Primary/Secondary Classroom,2991.0,...,NaN,2.0,583.0,NaN,NaN,NaN,NaN,NaN,D,1
874689,Lamb_education_Gabrielle,electricity,2017-02-17,231.1010,Lamb,706.0,5.0,Education,Primary/Secondary Classroom,2572.0,...,NaN,2.0,414.0,NaN,NaN,NaN,NaN,NaN,D,1
122020,Bull_education_Bryon,steam,2017-11-05,11.1630,Bull,891.0,9.0,Education,College Classroom,4723.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1078601,Panther_office_Daina,water,2017-01-11,113.5623,Panther,NaN,0.0,Office,Office,133.8,...,NaN,NaN,NaN,NaN,1983,NaN,NaN,NaN,NaN,1
850063,Lamb_education_Chet,electricity,2017-10-03,0.0024,Lamb,NaN,5.0,Education,Primary/Secondary Classroom,1078.0,...,NaN,2.0,235.0,NaN,NaN,NaN,NaN,NaN,C,1
501184,Eagle_office_Phyllis,hotwater,2017-03-25,348752.5851,Eagle,1297.0,14.0,Office,Office,1994.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
300306,Cockatoo_education_Latrice,electricity,2017-08-19,0.6750,Cockatoo,1345.0,15.0,Education,College Laboratory,1940.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1248926,Rat_office_Jill,electricity,2017-01-13,21423.2600,Rat,560.0,3.0,Office,Office,31587.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [11]:
# Inspecting the dataframe for a single building id 
merged_df[merged_df['building_id_kaggle'] == 636]

,building_id,meter,date,meter_reading,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,...,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating,value
0,Bear_education_Alfredo,electricity,2016-01-01,2.9050,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Bear_education_Alfredo,electricity,2016-01-02,2.7700,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Bear_education_Alfredo,electricity,2016-01-03,2.6725,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Bear_education_Alfredo,electricity,2016-01-04,4.5650,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Bear_education_Alfredo,electricity,2016-01-05,4.7825,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,Bear_education_Alfredo,electricity,2017-12-27,3.9825,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
727,Bear_education_Alfredo,electricity,2017-12-28,3.9975,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
728,Bear_education_Alfredo,electricity,2017-12-29,3.3650,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
729,Bear_education_Alfredo,electricity,2017-12-30,3.1825,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


# Missing Data

In [12]:
def determine_column_counts(df):
    columns = df.columns
    data_counts = pd.DataFrame(columns = ['column_name','column_type','num_rows','distinct_data', 'distinct_percent', 'missing_data', 'missing_percentage'])
    
    #For each column, calculate number and percentage of missing rows and distinct counts
    for col in columns:
        col_vals = df[col]
        total_data = len(col_vals)
        col_missing = sum(col_vals.isnull())
        missing_percent = round((col_missing / len(df[col])) * 100 , 3)
        col_distinct = (col_vals.nunique())
        distinct_percent = round((col_distinct / len(df[col])) * 100 , 3)

        data_counts.loc[len(data_counts)] = [col, col_vals.dtype, total_data, col_distinct, distinct_percent, col_missing, missing_percent]
        
    return(data_counts)

missing_data = determine_column_counts(merged_df)
missing_data

,column_name,column_type,num_rows,distinct_data,distinct_percent,missing_data,missing_percentage
0,building_id,object,1343578,924,0.069,0,0.000
1,meter,object,1343578,8,0.001,0,0.000
2,date,object,1343578,731,0.054,0,0.000
3,meter_reading,float64,1343578,874020,65.052,0,0.000
4,site_id,object,1343578,17,0.001,0,0.000
5,building_id_kaggle,float64,1343578,828,0.062,105264,7.835
6,site_id_kaggle,float64,1343578,16,0.001,9503,0.707
7,primaryspaceusage,object,1343578,2,0.000,0,0.000
8,sub_primaryspaceusage,object,1343578,15,0.001,0,0.000
9,sqm,float64,1343578,906,0.067,0,0.000


- On the basis of missing values and domain knowledge
- we shall drop the following columns since they are not directly consequential to our analysis

In [13]:
columns_to_drop = ['lat', 'lng','industry', 'subindustry',
                   'heatingtype', 'yearbuilt', 'date_opened', 'numberoffloors',
                   'occupants', 'energystarscore', 'eui', 'site_eui', 
                   'source_eui','leed_level', 'rating', 'value', 'site_id_kaggle', 'building_id_kaggle']

cleaned_df = merged_df.drop(columns = columns_to_drop)

In [14]:
cleaned_df.head()

,building_id,meter,date,meter_reading,site_id,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,timezone
0,Bear_education_Alfredo,electricity,2016-01-01,2.9050,Bear,Education,Education,609.8,6564.0,US/Pacific
1,Bear_education_Alfredo,electricity,2016-01-02,2.7700,Bear,Education,Education,609.8,6564.0,US/Pacific
2,Bear_education_Alfredo,electricity,2016-01-03,2.6725,Bear,Education,Education,609.8,6564.0,US/Pacific
3,Bear_education_Alfredo,electricity,2016-01-04,4.5650,Bear,Education,Education,609.8,6564.0,US/Pacific
4,Bear_education_Alfredo,electricity,2016-01-05,4.7825,Bear,Education,Education,609.8,6564.0,US/Pacific


In [15]:
missing_data_after_cleaning = determine_column_counts(cleaned_df)   
missing_data_after_cleaning

,column_name,column_type,num_rows,distinct_data,distinct_percent,missing_data,missing_percentage
0,building_id,object,1343578,924,0.069,0,0.0
1,meter,object,1343578,8,0.001,0,0.0
2,date,object,1343578,731,0.054,0,0.0
3,meter_reading,float64,1343578,874020,65.052,0,0.0
4,site_id,object,1343578,17,0.001,0,0.0
5,primaryspaceusage,object,1343578,2,0.000,0,0.0
6,sub_primaryspaceusage,object,1343578,15,0.001,0,0.0
7,sqm,float64,1343578,906,0.067,0,0.0
8,sqft,float64,1343578,906,0.067,0,0.0
9,timezone,object,1343578,6,0.000,0,0.0


In [17]:
os.makedirs("../data/cleaned", exist_ok=True)
cleaned_df.to_csv("../data/cleaned/cleaned_meters_meta.csv")

In [18]:
import gc
gc.collect()

0